# Создание предфинальной разметки

Для построения бейзлайна нужно сформировать некоторую базу текстов, так как все данные с википедии не поместятся в память и обучение не будет технически возможно. Для формирования сокращенной разметки будут выполнены следующие шаги:
* Будут взяты все тексты из 14 отобранных книг. Таких текстов не так много по сравнению с Вики, однако они ообогащены эмоциональной окраской, из-за чего в них чаще встречаются восклицательные и вопросительные знаки. Всего таких текстов чуть менее 30 тысяч, в сумме они содержат более миллиона слов. 
* Из данных Википедии будут взяты 830 тысяч абзацев.

In [90]:
import os
import tqdm
import pandas as pd
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
path_to_books = 'books/'

book_file_names = [i for i in os.listdir(path_to_books) if i != '.DS_Store']
len(book_file_names)

16

In [3]:
all_books_texts = []

for book_name in tqdm.tqdm(book_file_names):
    # в философии текст довольно сомнительного качества 
    if (book_name == 'bytie_i_vremya.txt') or (book_name == 'tak_govoril_zaratustra.txt'):
        continue
        
    with open(path_to_books + book_name, encoding='utf-8') as file:
        for line in file:
            line = line[:-1]
            
            # меняем тире
            line = re.sub('–', '—', line)
            
            # если реплика, то проверяем отстутсвие еще одного тире
            if (line[:2] == '— ') & (' — ' in line[2:]):
                continue
                
            elif line[:2] == '— ':
                line = line[2:]
                
            # меняем троеточие    
            line = re.sub('…', '...', line)

            all_books_texts.append(line)
            
len(all_books_texts)

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 55.65it/s]


27863

In [46]:
def clean_text_special_symbols(line):
    
    line = re.sub('– ', '', line)
    line = re.sub('— ', '', line)
#     line = re.sub('\(', '', line)
#     line = re.sub('\)', '', line)
    line = re.sub('"', '', line)
    line = line.lower()
    line = re.sub('\s+', ' ', line)
    
    return [token for token in line.split(' ') if token != '']

In [83]:
def create_labels(token_list):
    
    labels_puncts = ['.', ',', '?', '!', ';', '...', ':']
    labels = []
    for token in token_list:
        if (len(token) > 3) & (token[-3:] == '...'):
            labels.append('...')
        elif token[-1] in labels_puncts:
            labels.append(token[-1])
        else:
            labels.append('o')
    
    return labels

In [81]:
def get_markup(df_text):
    '''
    Функция, принимающая на вход датафрейм с колонкой текстов (text) и
    возвращающая датафрейм с соответствующими метками.
    1) Список очищенных токенов для обучения
    2) Метки основных знаков препинания:
    ['.', ',', '?', '!', ';', '...', ':', 'o']. (o = other)
    
    TODO:
    3) Метки кавычек и скобок:
    ['"_', '_"', ')', '(', 'o']
    4) Метки тире:
    ['—', 'o']
    
    Таким образом, для каждого текста будет возвращаться 4 списка
    '''
    # если сплитим по пробелу, то тире будет отдельным токеном
    # пока что удалим его и кавычки (для бейзлайна)
    df_text['tokens'] = df_text['text'].apply(clean_text_special_symbols)
    
    labels_puncts = ['.', ',', '?', '!', ';', '...', ':']
    df_text['labels'] = df_text['tokens'].apply(create_labels)
    
    df_text['tokens'] = df_text['tokens'].apply(lambda x: [re.sub("[^\w\s]", '', y) for y in x])
    
    return df_text


In [84]:
df_markup = pd.DataFrame()
df_markup['text'] = all_books_texts
df_markup = get_markup(df_markup)
df_markup

,text,tokens,labels
0,"Должен с сожалением сказать, что Фолио Клуб — ...","[должен, с, сожалением, сказать, что, фолио, к...","[o, o, o, ,, o, o, o, o, o, o, o, ., o, ,, o, ..."
1,"А между тем, когда я, всего какую-нибудь недел...","[а, между, тем, когда, я, всего, какуюнибудь, ...","[o, o, ,, o, ,, o, o, o, ,, o, o, o, o, ,, o, ..."
2,"В пять часов пополудни я, как было условлено, ...","[в, пять, часов, пополудни, я, как, было, усло...","[o, o, o, o, ,, o, o, ,, o, o, o, ,, o, o, o, ..."
3,Членами его были большей частью очень примечат...,"[членами, его, были, большей, частью, очень, п...","[o, o, o, o, o, o, o, ., o, o, o, o, o, ,, ,, ..."
4,"Был там также мистер Конволвулус Гондола, моло...","[был, там, также, мистер, конволвулус, гондола...","[o, o, o, o, o, ,, o, ,, o, o, .]"
...,...,...,...
27858,Так странно вдруг для Наташи показался этот ег...,"[так, странно, вдруг, для, наташи, показался, ...","[o, o, o, o, o, o, o, o, ,, ,, o, .]"
27859,Наташа в первый раз после многих дней заплакал...,"[наташа, в, первый, раз, после, многих, дней, ...","[o, o, o, o, o, o, o, o, o, o, o, o, ,, o, o, ..."
27860,Пьер тоже вслед за нею почти выбежал в передню...,"[пьер, тоже, вслед, за, нею, почти, выбежал, в...","[o, o, o, o, o, o, o, o, ,, o, o, o, o, ,, o, ..."
27861,"""Куда? — спросил себя Пьер. — Куда же можно ех...","[куда, спросил, себя, пьер, куда, же, можно, е...","[?, o, o, ., o, o, o, o, ?, o, o, o, o, o, ?, ..."


In [70]:
np.random.seed(999)

paths_to_folders = os.listdir('all_articles_edited/')
wiki_texts = []

for file_name in tqdm.tqdm(paths_to_folders):
    folder_texts = []
    with open('all_articles_edited/' + file_name, 'r') as file:
        if file_name == '.DS_Store':
            continue
            
        for line in file:
            line = line[:-1]
            # убираем тезнические символы и заголовки
            if ('#' in line) or (' ,' in line)  or (' ;' in line)\
                or (' .' in line) or (len(line.split(' ')) <= 4):
                continue
            folder_texts.append(line)
            
        wiki_texts += np.random.choice(folder_texts, size=10000, replace=False).tolist() 

100%|███████████████████████████████████████████| 83/83 [04:34<00:00,  3.30s/it]


In [85]:
len(wiki_texts)

830000

In [86]:
wiki_texts[:5]

['На драфте НХЛ 1998 года был выбран в 4 раунде под общим 108 номером командой "Калгари Флеймз". В 2003—2004 годах выступал за "Лас-Вегас Рэнглерс". 10 августа 2005 года как неограниченно свободный агент подписал контракт с "Питтсбург Пингвинз". 4 октября 2006 года был забран на вейвере "Ванкувер Кэнакс". 1 июля 2007 года как неограниченно свободный агент подписал контракт с "Питтсбург Пингвинз".',
 'Категория лица имени существительного выражается личными показателями, совпадающими с личными местоимениями. В двойственном и множественном числе личные показатели ставятся непосредственно после основ существительных. В первом и втором лицах единственного числа между основой существительного и показателем вставляется "j". Если основа оканчивается на согласный, вставляется "i". Третье лицо не имеет специальных форм личных показателей. Личные показатели употребляются только в номинативе. В косвенных падежах они заменяются падежными показателями.',
 'В 2007 году, в результате комплексной пров

In [87]:
df_markup_wiki = pd.DataFrame()
df_markup_wiki['text'] = wiki_texts
df_markup_wiki = get_markup(df_markup_wiki)
df_markup_wiki

,text,tokens,labels
0,На драфте НХЛ 1998 года был выбран в 4 раунде ...,"[на, драфте, нхл, 1998, года, был, выбран, в, ...","[o, o, o, o, o, o, o, o, o, o, o, o, o, o, o, ..."
1,Категория лица имени существительного выражает...,"[категория, лица, имени, существительного, выр...","[o, o, o, o, o, o, ,, o, o, o, ., o, o, o, o, ..."
2,"В 2007 году, в результате комплексной проверки...","[в, 2007, году, в, результате, комплексной, пр...","[o, o, ,, o, o, o, o, o, o, o, ,, o, o, o, o, ..."
3,Во главе его стоял Адольф Гоздава-Реутт (псевд...,"[во, главе, его, стоял, адольф, гоздавареутт, ...","[o, o, o, o, o, o, o, ,, o, o, o, o, o, o, o, ..."
4,В 1847 году поступил в Петербургское строитель...,"[в, 1847, году, поступил, в, петербургское, ст...","[o, o, o, o, o, o, o, ., o, o, o, o, o, o, o, ..."
...,...,...,...
829995,Был противником большевизма. 31 марта 1918 год...,"[был, противником, большевизма, 31, марта, 191...","[o, o, ., o, o, o, o, o, o, o, o, o, o, o, :, ..."
829996,Tischeria quercitella — вид одноцветных молей-...,"[tischeria, quercitella, вид, одноцветных, мол...","[o, o, o, o, o, o, o, ., o, o, o, o, o, ., o, ..."
829997,"По данным 1914 года, в деревне проживало 46 че...","[по, данным, 1914, года, в, деревне, проживало...","[o, o, o, ,, o, o, o, o, o, o, o, o, o, ., o, ..."
829998,"Биюк-Узень (также ""исток реки Кача"", ""Хавалы́х...","[биюкузень, также, исток, реки, кача, хавалыхд...","[o, o, o, o, ,, o, ,, o, o, o, o, ,, o, o, o, ..."


Как видно из последних примеров, тире сейчас проглатывается.

In [88]:
final_markup = pd.concat([df_markup, df_markup_wiki]).reset_index(drop=True)
labels.append(token[-1]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857863 entries, 0 to 857862
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    857863 non-null  object
 1   tokens  857863 non-null  object
 2   labels  857863 non-null  object
dtypes: object(3)
memory usage: 19.6+ MB


In [89]:
markup_stats = Counter()

for row in tqdm.tqdm(final_markup.values):
    markup_stats += Counter(row[2])

markup_stats 

100%|███████████████████████████████| 857863/857863 [00:04<00:00, 213838.82it/s]


Counter({'o': 30445317,
         ',': 2748598,
         '.': 2496043,
         ';': 77286,
         ':': 93630,
         '!': 8703,
         '?': 8793,
         '...': 2769})

In [91]:
data_dev, data_test = train_test_split(final_markup, test_size=0.1, random_state=999)
data_train, data_val = train_test_split(data_dev, test_size=0.2, random_state=999)

data_train.to_csv('train.csv', index=False)
data_val.to_csv('val.csv', index=False)
data_test.to_csv('test.csv', index=False)

In [92]:
markup_stats = Counter()

for row in tqdm.tqdm(data_train.values):
    markup_stats += Counter(row[2])

markup_stats 

100%|███████████████████████████████| 617660/617660 [00:02<00:00, 206987.01it/s]


Counter({'o': 21911068,
         ',': 1977130,
         '.': 1796203,
         ':': 67245,
         ';': 55604,
         '?': 6314,
         '!': 6193,
         '...': 1948})

In [93]:
markup_stats = Counter()

for row in tqdm.tqdm(data_val.values):
    markup_stats += Counter(row[2])

markup_stats 

100%|███████████████████████████████| 154416/154416 [00:00<00:00, 208356.03it/s]


Counter({'o': 5492890,
         '.': 450557,
         ',': 496778,
         ':': 16996,
         ';': 14007,
         '?': 1625,
         '...': 553,
         '!': 1647})

In [94]:
markup_stats = Counter()

for row in tqdm.tqdm(data_test.values):
    markup_stats += Counter(row[2])

markup_stats 

100%|█████████████████████████████████| 85787/85787 [00:00<00:00, 204956.53it/s]


Counter({'o': 3041359,
         ',': 274690,
         '.': 249283,
         ':': 9389,
         ';': 7675,
         '?': 854,
         '...': 268,
         '!': 863})